# Predict accuracy

### Setup

activate npx

In [3]:
%load_ext autoreload
%autoreload 2
import os
import spikeinterface as si

PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/sfn_2023"
os.chdir(PROJ_PATH)

from src.nodes.utils import get_config

EXPERIMENT = "silico_horvath"
DATE = "probe_1"
data_conf, _ = get_config(EXPERIMENT, DATE).values()
SORTING_PATH_1 = data_conf["sorting"]["sorters"]["kilosort3"]["output"]
GT_SORTING_PATH_1 = data_conf["sorting"]["simulation"]["ground_truth"]["output"]
RECORDING_PATH_1 = data_conf["probe_wiring"]["input"]
BLUECONFIG_PATH_1 = data_conf["dataeng"]["blueconfig"]

DATE = "probe_2"
data_conf, _ = get_config(EXPERIMENT, DATE).values()
SORTING_PATH_2 = data_conf["sorting"]["sorters"]["kilosort3"]["output"]
GT_SORTING_PATH_2 = data_conf["sorting"]["simulation"]["ground_truth"]["output"]
RECORDING_PATH_2 = data_conf["probe_wiring"]["input"]
BLUECONFIG_PATH_2 = data_conf["dataeng"]["blueconfig"]

DATE = "probe_3"
data_conf, _ = get_config(EXPERIMENT, DATE).values()
SORTING_PATH_3 = data_conf["sorting"]["sorters"]["kilosort3"]["output"]
GT_SORTING_PATH_3 = data_conf["sorting"]["simulation"]["ground_truth"]["output"]
RECORDING_PATH_3 = data_conf["probe_wiring"]["input"]
BLUECONFIG_PATH_3 = data_conf["dataeng"]["blueconfig"]

2023-11-06 17:55:41,169 - root - utils.py - get_config - INFO - Reading experiment config.
2023-11-06 17:55:41,202 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-11-06 17:55:41,203 - root - utils.py - get_config - INFO - Reading experiment config.
2023-11-06 17:55:41,217 - root - utils.py - get_config - INFO - Reading experiment config. - done
2023-11-06 17:55:41,218 - root - utils.py - get_config - INFO - Reading experiment config.
2023-11-06 17:55:41,238 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [5]:
# get sorting accuracy for each true unit
# Sorting1 = si.load_extractor(GT_SORTING_PATH_1)

# get all their properties
# Sorting1.get_property("sorting_accuracy")

# SILICO
silico_paths = [GT_SORTING_PATH_1, GT_SORTING_PATH_2, GT_SORTING_PATH_3]

firing_rate_all = []
layer_all = []
depth_all = []
unit_id_all = []
KSLabel_all = []
amplitude_all = []
false_positives_all = []

for ix, sorted_path in enumerate(silico_paths):
    Sorting = si.load_extractor(sorted_path)
    # record
    unit_id_all += Sorting.unit_ids.tolist()
    firing_rate_all += Sorting.get_property("firing_rates").astype(np.float32).tolist()
    layer_all += Sorting.get_property("layer").tolist()
    KSLabel_all += Sorting.get_property("KSLabel").tolist()
    amplitude_all += Sorting.get_property("Amplitude").tolist()
    false_positives_all += Sorting.get_property("false_positives").tolist()
    depth_all += len(Sorting.unit_ids) * [ix + 1]


# store in dataframe
df_silico = pd.DataFrame(
    np.array(
        [
            depth_all,
            layer_all,
            firing_rate_all,
            KSLabel_all,
            amplitude_all,
            false_positives_all,
        ]
    ).T,
    index=unit_id_all,
    columns=[
        "depth",
        "layer",
        "firing_rate",
        "kslabel",
        "amplitude",
        "false_positives",
    ],
)

# run logistic regression or regression tree

array([0.05442177, 0.00290698, 0.13461538, 0.17037037, 0.84482759,
       0.85714286, 0.36956522, 0.58064516, 0.05426357, 0.09615385,
       0.01904762, 0.06965174, 0.03100775, 0.01612903, 0.04666667,
       0.00854701, 0.10404624, 0.        , 0.20472441, 0.3125    ,
       0.00534759, 0.1125    , 0.04098361, 0.00265957, 0.00952381,
       0.06930693, 0.0027027 , 0.00900901, 0.11428571, 0.15384615,
       0.03418803, 0.0483871 , 0.02531646, 0.13333333, 1.        ,
       0.01840491, 0.01242236, 0.01290323, 0.14204545, 0.08823529,
       0.69230769, 0.00719424, 0.00632911, 0.16981132, 0.04950495,
       0.056     , 0.04310345, 0.64166667, 0.02150538, 0.03333333,
       0.06862745, 0.00847458, 0.03937008, 0.064     , 0.00869565,
       0.02884615, 0.02469136, 0.16      , 0.01360544, 0.01219512,
       0.0201005 , 0.02205882, 0.16666667, 0.06896552, 0.15079365,
       0.13043478, 0.02325581, 0.04444444, 0.00312012, 0.07518797,
       0.136     , 0.01086957, 0.04032258, 0.09625668, 0.06779